In [ ]:
import pandas as pd
import numpy as np
import os
import os.path
from os.path import join

In [ ]:
df_ref = pd.read_csv(r'T:\ABM\ABM_FY22\RSM\VisionEval\Model\Update_Automations\SrcOnSanVE\geo\geo.csv')
bzone_ref = df_ref[['Bzone','Marea']]

In [ ]:
def assignTransitService_marea (bzone_ref):
    bzone_ref.columns = ['taz', 'Marea']
    input_path = r"T:\ABM\ABM_FY22\RSM\VisionEval\Model\Update_Automations\SrcOnSanVE\modules\VETransportSupply\data" 
    years = ['2016', '2035build']
    data = []
    for year in years:
        stop = pd.read_csv(os.path.join(input_path, year, 'routeMarea.csv'))
        VMT = pd.read_csv(os.path.join(input_path, year, 'transitVMT.csv'))
        stop = stop.merge(bzone_ref, right_on = "taz", left_on = "ZONE", how = "left")
        route_list = list(stop.Route.unique())
        VMT_route = VMT.loc[VMT['route'].isin(route_list)]
        #convert daily to annual VMT
        if year[:4] == '2016':
            VMT_route['Avmt'] = VMT_route['vmt'] * 261 + VMT_route['vmt'] * 0.5 * 105
        if year[:4] == '2035':
            VMT_route['Avmt'] = VMT_route['vmt'] * 261 + VMT_route['vmt'] * 0.5 * 104
        
        conditions = [(VMT_route['mode'].isin(['Arterial Rapid','Express Bus','Freeway Rapid', 'Premium Express Bus'])),
                    (VMT_route['mode'] == 'Commuter Rail'),
                    (VMT_route['mode'] == 'Light Rail'),
                    (VMT_route['mode'] == 'Local Bus'),
                    (VMT_route['mode'] == 'Commuter heavy rail')]
        choices = ['RBRevMi', 'CRRevMi', 'SRRevMi', 'MBRevMi', 'HRRevMi']
        VMT_route['VE'] = np.select(conditions, choices, default=np.nan)
        df = VMT_route.pivot_table(values='Avmt', columns='VE', aggfunc = np.sum).reset_index()
        if year[:4] == '2016':
            df['HRRevMi'] = 0
        df['Year'] = year[:4]
        data.append(df)
    df_final = pd.concat(data)
    #add DR, VP and MG with 0 mile
    df_final['DRRevMi'] = 0
    df_final['VPRevMi'] = 0
    df_final['MGRevMi'] = 0
    df_final['Geo'] = 'SD-Urban-TransitService'
    df_final = df_final[['Geo', 'Year', 'DRRevMi', 'MBRevMi', 'RBRevMi', 'VPRevMi', 'MGRevMi', 'SRRevMi', 'HRRevMi', 'CRRevMi']]
    df_final.to_csv('marea_transit_service.csv', index=False)

In [ ]:
def assignTransitService_bzone (bzone_ref):
    bzone_ref.columns = ['taz', 'Marea']
    input_path = r"T:\ABM\ABM_FY22\RSM\VisionEval\Model\Update_Automations\SrcOnSanVE\modules\VETransportSupply\data" 
    years = ['2016', '2035build']
    data = []
    for year in years:
        stop = pd.read_csv(os.path.join(input_path, year, 'routeMarea.csv'))
        trrt = pd.read_csv(os.path.join(input_path, year, 'trrt.csv'))
        trrt['PM_fre'] = 0
        trrt.loc[trrt['PM_Headway'] != 0, 'PM_fre'] = 1/(trrt['PM_Headway']/60)
        stop = stop.merge(trrt, on = "Route_Name", how = "left")
        
        df = stop.pivot_table(values='PM_fre', index='ZONE', aggfunc = np.sum).reset_index()      
        df2 = bzone_ref.merge(df, right_on = "ZONE", left_on = "taz", how = "left")
        df2 = df2.fillna(0)
        df2['Year'] = year[:4]
        data.append(df2)
    df_final = pd.concat(data)
    df_final = df_final.rename({'taz': 'Geo', 'PM_fre': 'D4c'}, axis=1)
    df_final = df_final[['Geo', 'Year', 'D4c']]
    df_final.to_csv('bzone_transit_service.csv', index=False)